## prepare envirvonment

In [1]:
%%configure -f
{
    "driverMemory": "1G", 
    "driverCores": 2,
    "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.files": "s3://pi-pw2-pareto/xgboost_native_model_from_xgb_ESRD_production.pkl-0.bin"
         }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
161,application_1649090359691_0192,pyspark,idle,Link,Link,gasante,
163,application_1649090359691_0195,pyspark,idle,Link,Link,whensel,
164,application_1649090359691_0196,pyspark,idle,Link,Link,espalding,


In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
161,application_1649090359691_0192,pyspark,idle,Link,Link,gasante,
163,application_1649090359691_0195,pyspark,idle,Link,Link,whensel,
164,application_1649090359691_0196,pyspark,idle,Link,Link,espalding,


In [3]:
# Load Pyspark packages
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
import xgboost as xgb
import shap

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
166,application_1649090359691_0198,pyspark,idle,Link,Link,nstevenson,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'xgboost'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'xgboost'



In [4]:
# Import Python packages
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
#import joblib
import numpy as np
import random
import pickle
#from sklearn import linear_model, model_selection, ensemble, svm, naive_bayes, preprocessing
#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import getpass
import sys
import urllib
import time
from datetime import date
#from future.standard_library import install_aliases
import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
import datetime as dt
import shap

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
%%local
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
#import joblib
import numpy as np
import random
import pickle
#from sklearn import linear_model, model_selection, ensemble, svm, naive_bayes, preprocessing
#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import getpass
import sys
import sqlalchemy
import urllib
import time
from datetime import date
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
#import xgboost as xgb
import datetime as dt
#import shap
#from matplotlib import pyplot as plt

In [6]:
print(shap.__version__)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.39.0

In [9]:
%%local
import getpass
password = getpass.getpass()

 ············


In [10]:
%%send_to_spark -i password -t str -n password

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'password' as 'password' to Spark kernel

## pull data from sql into pyspark

In [11]:
%%spark

hostname = "HW2WIPSQL10.HEALTHSCAPEADVISORS.NET"
database = "MartinsPoint_1315"
userName = "nstevenson"
port = 1433

url = f"jdbc:sqlserver://{hostname}:{port};database={database}"
connectionProperties = {
    "integratedSecurity" : "true",
    "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver",
    "authenticationScheme" :"NTLM",
    "domainName" : "healthscapeadvisors.net",
    "userName"   : "nstevenson",
    "password"   : password
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
dfspark_esrd = spark.read.jdbc(url=url, table="TEST_ESRDMod_Merge6", properties=connectionProperties)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
query = """(
SELECT DISTINCT a.insuredMemberIdentifier, b.ESRD_Flag
FROM TEST_ESRDMod_Merge6 a
INNER JOIN (
SELECT DISTINCT x.HSA_ID, MAX(x.ESRD) AS ESRD_Flag
FROM BackendMMRTransform x
GROUP BY x.HSA_ID
) b
ON b.HSA_ID = a.insuredMemberIdentifier

)"""

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
dfspark_flag = spark.read.format("jdbc").option("url", url) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("integratedSecurity", "true").option("authenticationScheme", "NTLM").option("domainName", "healthscapeadvisors.net") \
    .option("userName", 'nstevenson').option("password", password) \
    .option("query", query).load()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## etl data in pyspark

In [15]:
dfspark_esrd = dfspark_esrd.dropDuplicates()
dfspark_esrd = dfspark_esrd.dropDuplicates(subset=['insuredMemberIdentifier', 'insuredMemberGenderCode'])
dfspark_esrd = dfspark_esrd.withColumnRenamed('Age','Age1')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
dfspark_esrd = dfspark_esrd.withColumn('BTC_721', dfspark_esrd['BTC_721'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('BTC_722', dfspark_esrd['BTC_722'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('BTC_723', dfspark_esrd['BTC_723'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('BTC_724', dfspark_esrd['BTC_724'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('BTC_727', dfspark_esrd['BTC_727'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('BTC_728', dfspark_esrd['BTC_728'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('GPI_30905225', dfspark_esrd['GPI_30905225'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('GPI_52800020', dfspark_esrd['GPI_52800020'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('GPI_52800070', dfspark_esrd['GPI_52800070'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('GPI_78133000', dfspark_esrd['GPI_78133000'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('CPTOut90951_90962', dfspark_esrd['CPTOut90951_90962'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('CPTHome90963_90966', dfspark_esrd['CPTHome90963_90966'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('CPTPartial90967_90970', dfspark_esrd['CPTPartial90967_90970'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('ESRD_Neph', dfspark_esrd['ESRD_Neph'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('CKD_post_ESRD', dfspark_esrd['CKD_post_ESRD'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('ESRD_Enroll_Diff', dfspark_esrd['ESRD_Enroll_Diff'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('Diag_CPT_Diff', dfspark_esrd['Diag_CPT_Diff'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('ESRD_Neph_Only', dfspark_esrd['ESRD_Neph_Only'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('CPT_Enroll_Diff', dfspark_esrd['CPT_Enroll_Diff'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('BTC_CPT_Diff', dfspark_esrd['BTC_CPT_Diff'].cast(FloatType()))
dfspark_esrd = dfspark_esrd.withColumn('insuredMemberGenderCode', dfspark_esrd['insuredMemberGenderCode'].cast(BooleanType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
dfspark_esrd = dfspark_esrd.withColumn('Dialysis_and_CKD5', lit(0))
dfspark_esrd = dfspark_esrd.withColumn('Dialysis_and_ESRD', lit(0))

dfspark_esrd = dfspark_esrd.withColumn("Dialysis_and_CKD5", \
                       when(((dfspark_esrd['`Dialysis_V45.11_Tot`'] > 0)
            | (dfspark_esrd["`Dialysis_V56.0_Z49.01_Tot`"] > 0) 
            | (dfspark_esrd["`Dialysis_V56.1_Z49.02_Tot`"] > 0))
            & (dfspark_esrd["`CKD5_585.5_N18.5_Tot`"] > 0), 1).otherwise(0))

dfspark_esrd = dfspark_esrd.withColumn("Dialysis_and_ESRD", \
                       when(((dfspark_esrd['`Dialysis_V45.11_Tot`'] > 0)
            | (dfspark_esrd["`Dialysis_V56.0_Z49.01_Tot`"] > 0) 
            | (dfspark_esrd["`Dialysis_V56.1_Z49.02_Tot`"] > 0))
            & (dfspark_esrd["`ESRD_585.6_N18.6_Tot`"] > 0), 1).otherwise(0))

dfspark_esrd = dfspark_esrd.withColumn("insuredMemberGenderCode", \
                       when((dfspark_esrd['insuredMemberGenderCode'] == 'M'), 0).otherwise(
                                            when((dfspark_esrd['insuredMemberGenderCode'] == 'F'), 1).otherwise(0)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
dfspark_flag = dfspark_flag.withColumn("ESRD_Flag", \
                       when((dfspark_flag['ESRD_Flag'] == 'Y'), 1).otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
dfspark_flag.groupby('ESRD_Flag').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+
|ESRD_Flag|count|
+---------+-----+
|        1|  160|
|        0| 8714|
+---------+-----+

## modeling process

In [20]:
features = ['insuredMemberGenderCode',
     'BTC_Enroll_Diff',
     'BTC_721',
     'BTC_722',
     'BTC_723',
     'BTC_724',
     'BTC_727',
     'BTC_728',
     'ESRD_Enroll_Diff',
     'CKD_Enroll_Diff',
     'CKD_post_ESRD',
     'ESRD_Neph',
     'CKD_Neph',
     'ESRD_Neph_Only',
     'ESRD_585.6_N18.6_Tot',
     'CKD5_585.5_N18.5_Tot',
     'CKD4_585.4_N18.4_Tot',
     'CKD5_585.3_N18.3_Tot',
     'CKD5_585.2_N18.2_Tot',
     'CKD5_585.1_N18.1_Tot',
     'Dialysis_V45.11_Tot',
     'Dialysis_V56.0_Z49.01_Tot',
     'Dialysis_V56.1_Z49.02_Tot',
     'AKI_Tot',
     'GPI_30905225',
     'GPI_52800020',
     'GPI_52800070',
     'GPI_78133000',
     'CPT_Enroll_Diff',
     'CPTOut90951_90962',
     'CPTHome90963_90966',
     'CPTPartial90967_90970',
     'BTC_Diag_Diff',
     'BTC_CPT_Diff',
     'Diag_CPT_Diff',
     'Age1',
     'Dialysis_and_CKD5',
     'Dialysis_and_ESRD']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df = dfspark_esrd.withColumn('insuredMemberIdentifier', dfspark_esrd['insuredMemberIdentifier'].cast(StringType()))
df = df.select('insuredMemberIdentifier')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
dfspark_esrd = dfspark_esrd.join(dfspark_flag, 'insuredMemberIdentifier')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
xgbc = xgb.XGBClassifier()
xgbc.load_model("xgboost_native_model_from_xgb_ESRD_production.pkl-0.bin")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[15:09:15] WARNING: /tmp/pip-build-qbfnnz8r/xgboost/build/temp.linux-x86_64-3.7/xgboost/src/learner.cc:790: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility

In [15]:
xgbc.save_model("thrisha_test.bin")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
dfspark_esrd_model = dfspark_esrd.select("insuredMemberGenderCode",
     "BTC_Enroll_Diff",
     "BTC_721",
     "BTC_722",
     "BTC_723",
     "BTC_724",
     "BTC_727",
     "BTC_728",
     "ESRD_Enroll_Diff",
     "CKD_Enroll_Diff",
     "CKD_post_ESRD",
     "ESRD_Neph",
     "CKD_Neph",
     "ESRD_Neph_Only",
     '`ESRD_585.6_N18.6_Tot`',
     '`CKD5_585.5_N18.5_Tot`',
     '`CKD4_585.4_N18.4_Tot`',
     '`CKD5_585.3_N18.3_Tot`',
     '`CKD5_585.2_N18.2_Tot`',
     '`CKD5_585.1_N18.1_Tot`',
     '`Dialysis_V45.11_Tot`',
     '`Dialysis_V56.0_Z49.01_Tot`',
     '`Dialysis_V56.1_Z49.02_Tot`',
     "AKI_Tot",
     "GPI_30905225",
     "GPI_52800020",
     "GPI_52800070",
     "GPI_78133000",
     "CPT_Enroll_Diff",
     "CPTOut90951_90962",
     "CPTHome90963_90966",
     "CPTPartial90967_90970",
     "BTC_Diag_Diff",
     "BTC_CPT_Diff",
     "Diag_CPT_Diff",
     "Age1",
     "Dialysis_and_CKD5",
     "Dialysis_and_ESRD")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'dfspark_esrd' is not defined
Traceback (most recent call last):
NameError: name 'dfspark_esrd' is not defined



In [25]:
dfspark_esrd_model.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(insuredMemberGenderCode=0, BTC_Enroll_Diff=None, BTC_721=None, BTC_722=None, BTC_723=None, BTC_724=None, BTC_727=None, BTC_728=None, ESRD_Enroll_Diff=None, CKD_Enroll_Diff=593, CKD_post_ESRD=None, ESRD_Neph=0.0, CKD_Neph=0, ESRD_Neph_Only=0.0, ESRD_585.6_N18.6_Tot=0, CKD5_585.5_N18.5_Tot=0, CKD4_585.4_N18.4_Tot=0, CKD5_585.3_N18.3_Tot=2, CKD5_585.2_N18.2_Tot=0, CKD5_585.1_N18.1_Tot=0, Dialysis_V45.11_Tot=0, Dialysis_V56.0_Z49.01_Tot=0, Dialysis_V56.1_Z49.02_Tot=0, AKI_Tot=0, GPI_30905225=0.0, GPI_52800020=0.0, GPI_52800070=0.0, GPI_78133000=0.0, CPT_Enroll_Diff=None, CPTOut90951_90962=None, CPTHome90963_90966=None, CPTPartial90967_90970=None, BTC_Diag_Diff=None, BTC_CPT_Diff=None, Diag_CPT_Diff=None, Age1=73, Dialysis_and_CKD5=0, Dialysis_and_ESRD=0)

In [26]:
df_model=dfspark_esrd_model.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
xgbc.predict_proba(dfspark_esrd_model.toPandas())[:,1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([0.00219146, 0.00153974, 0.006097  , ..., 0.00154413, 0.0130042 ,
       0.00020546], dtype=float32)

In [28]:
df=dfspark_esrd.toPandas()[['insuredMemberIdentifier']]
df['Predict']=xgbc.predict_proba(dfspark_esrd_model.toPandas())[:,1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     insuredMemberIdentifier   Predict
0                1A14UC6DG85  0.002191
1                1AE3KH7NQ04  0.001540
2                1AH2UQ9UR28  0.006097
3                1AW0V53HF49  0.026871
4                1CU2FT1QR97  0.000249
...                      ...       ...
8869             9FY3RR4DF01  0.002874
8870             9JF4UU2KY82  0.000213
8871             9QV8U29EM44  0.001544
8872             9U79G07QP27  0.013004
8873             9VK7H56UP20  0.000205

[8874 rows x 2 columns]

In [30]:
start = time.time()
explainerXGB = shap.TreeExplainer(xgbc)
end = time.time()
print(end - start)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7589800357818604

In [31]:
df_esrd_pandas = dfspark_esrd_model.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
start = time.time()
shap_values_XGB_test = explainerXGB.shap_values(df_esrd_pandas)
end = time.time()
print(end - start)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4.325282573699951

In [33]:
df_current=df_esrd_pandas.copy()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
start = time.time()
for idx, row in df_current.iterrows():
    row2=df_current.iloc[[idx]]
    shap_values = explainerXGB.shap_values(row2[features])
    #which_class = df_current.at[idx,'y_pred']
    df=pd.DataFrame(data=shap_values, columns=df_current[features].columns, index=[0])
    df1 = df.transpose()
    df1.sort_values(by=0, ascending=False, inplace = True)
    #df1.drop(['year', 'MMRMonthKey_x'], inplace=True)
    for i in range(0,5):
        feature = df1.index.values[i]        
        value=df_current.at[idx,feature]
        if pd.isnull(value):
            value='unknown'
        feat_val=str(str(feature) + " = " + str(value))
        feat_insert="Feature_{0}".format(i+1)
        df_current.at[idx,feat_insert]=feat_val
end = time.time()
print(end-start)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

106.92163133621216

In [42]:
shap_values_XGB_test

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 6.7322724e-02,  2.6095415e-02,  4.4856690e-02, ...,
        -2.3927472e-03,  6.8514462e-05,  1.2972498e-02],
       [ 7.5177960e-02, -5.2355215e-02,  5.3355447e-03, ...,
        -8.2429403e-01, -2.0075794e-03,  1.2209073e-02],
       [-5.5508822e-02, -1.8135859e-02,  2.3956191e-02, ...,
        -6.1215156e-01, -1.3121758e-03,  1.4973300e-02],
       ...,
       [-1.0125050e-03,  3.2243479e-02,  3.8200095e-02, ...,
         6.8931282e-02,  1.8247191e-03,  1.6935965e-02],
       [-3.4428257e-02,  4.5973323e-02,  2.6642224e-02, ...,
         3.6768466e-01,  3.3664829e-03,  2.2450807e-02],
       [-1.4953499e-01,  2.0841235e-02,  5.0572913e-02, ...,
        -5.7612855e-02, -7.4988994e-04,  2.7655648e-02]], dtype=float32)

In [45]:
df_current.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   insuredMemberGenderCode  ...                    Feature_5
0                        0  ...                  AKI_Tot = 0
1                        1  ...    BTC_Enroll_Diff = unknown
2                        0  ...  insuredMemberGenderCode = 0
3                        1  ...    BTC_Enroll_Diff = unknown
4                        1  ...        Dialysis_and_ESRD = 0

[5 rows x 43 columns]

In [47]:
df=dfspark_esrd.toPandas()[['insuredMemberIdentifier']]
df['Predict']=xgbc.predict_proba(dfspark_esrd_model.toPandas())[:,1]
df['Feature_1']=df_current['Feature_1']
df['Feature_2']=df_current['Feature_2']
df['Feature_3']=df_current['Feature_3']
df['Feature_4']=df_current['Feature_4']
df['Feature_5']=df_current['Feature_5']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df.sort_values(by=['Predict'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     insuredMemberIdentifier   Predict                    Feature_1  \
6084             4F53DR3UA74  0.000045            BTC_723 = unknown   
27               1X14P21EE63  0.000061            BTC_723 = unknown   
95               3UN4EJ5XV94  0.000065            BTC_723 = unknown   
6911             7NM8NP2NP38  0.000068            BTC_723 = unknown   
8549             4HA2XG2UX87  0.000079  insuredMemberGenderCode = 0   
...                      ...       ...                          ...   
1435             1VY6HF2GA30  0.986265                BTC_722 = 0.0   
7767             2GE8KP4UF46  0.986752                BTC_722 = 0.0   
1275             5XX5TT3XD89  0.986887                BTC_722 = 0.0   
841              4RC8UX7KC31  0.990470                BTC_722 = 0.0   
2967             5PU7XH6QT50  0.991807       ESRD_Enroll_Diff = 8.0   

                    Feature_2                     Feature_3  \
6084          ESRD_Neph = 0.0         Dialysis_and_ESRD = 0   
27            ESRD_Ne

In [60]:
pd.set_option('display.max_columns',None)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…